![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BartForZeroShot.ipynb)

# Import ONNX BartForZeroShot models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- The BartForZeroShot model was introduced in `Spark NLP 5.1.0 and requires Spark version 3.4.1 and up.`
- Official models are supported, but not all custom models may work.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

- Hugging Face has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to export ONNX models directly via the CLI with `!optimum-cli export onnx --model MODEL_NAME EXPORT_PATH`.
- We'll use the [facebook/bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli) model from HuggingFace as an example and export it with the `optimum-cli`.

In [2]:
MODEL_NAME = "facebook/bart-large-mnli"
EXPORT_PATH = f"export_onnx/{MODEL_NAME}"

!optimum-cli export onnx --model {MODEL_NAME} {EXPORT_PATH}

2025-06-07 04:47:08.356883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749271628.398959    1440 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749271628.415968    1440 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749271628.448297    1440 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749271628.448358    1440 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749271628.448369    1440 computation_placer.cc:177] computation placer alr

Let's have a look inside these two directories and see what we are dealing with:

In [7]:
!ls -l {EXPORT_PATH}

total 1596384
drwxr-xr-x 2 root root       4096 Jun  7 04:49 assets
-rw-r--r-- 1 root root       1237 Jun  7 04:47 config.json
-rw-r--r-- 1 root root 1630314894 Jun  7 04:48 model.onnx
-rw-r--r-- 1 root root        279 Jun  7 04:47 special_tokens_map.json
-rw-r--r-- 1 root root       1243 Jun  7 04:47 tokenizer_config.json
-rw-r--r-- 1 root root    3558642 Jun  7 04:48 tokenizer.json
-rw-r--r-- 1 root root     798293 Jun  7 04:47 vocab.json


- We need to move `vocab.txt` and `merges.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find them. However, before doing that, we first need to convert Hugging Face’s `vocab.json` into a plain `vocab.txt` format, as Spark NLP does not support the JSON format.
- Additionally, we need to extract the `labels` and their corresponding `ids` from the model's config. This mapping will be saved as `labels.txt` inside the same `assets` folder.

In [4]:
!mkdir -p {EXPORT_PATH}/assets

from transformers import AutoConfig
config = AutoConfig.from_pretrained(MODEL_NAME)
labels = [v for _, v in sorted(config.id2label.items())]
with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join(labels))

import json
with open(f"{EXPORT_PATH}/vocab.json") as f_in, open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as f_out:
    for key in json.load(f_in):
        print(key, file=f_out)

!mv {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets/

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
!ls -l {EXPORT_PATH}/assets

total 1596384
drwxr-xr-x 2 root root       4096 Jun  7 04:49 assets
-rw-r--r-- 1 root root       1237 Jun  7 04:47 config.json
-rw-r--r-- 1 root root 1630314894 Jun  7 04:48 model.onnx
-rw-r--r-- 1 root root        279 Jun  7 04:47 special_tokens_map.json
-rw-r--r-- 1 root root       1243 Jun  7 04:47 tokenizer_config.json
-rw-r--r-- 1 root root    3558642 Jun  7 04:48 tokenizer.json
-rw-r--r-- 1 root root     798293 Jun  7 04:47 vocab.json


In [8]:
!cat {EXPORT_PATH}/assets/labels.txt

contradiction
neutral
entailment

Voila! We have our `vocab.txt` and `merges.txt` inside assets directory, along with the extracted labels saved in `labels.txt`.

## Import and Save BartForZeroShotClassification  in Spark NLP


Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [ ]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 37.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [10]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BartForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BartForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [11]:
from sparknlp.annotator import BartForZeroShotClassification

zero_shot_classifier = BartForZeroShotClassification.loadSavedModel(
    EXPORT_PATH,
    spark
    )\
  .setInputCols(["document", "token"]) \
  .setOutputCol("class") \
  .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [12]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [17]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your BartForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [16]:
! ls -l {EXPORT_PATH}_spark_nlp

total 1592360
-rw-r--r-- 1 root root 1630563798 Jun  7 04:54 bart_classification_onnx
drwxr-xr-x 5 root root       4096 Jun  7 04:53 fields
drwxr-xr-x 2 root root       4096 Jun  7 04:53 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForSequenceClassification model 😊

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [15]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import BartForZeroShotClassification, Tokenizer

classifier = BartForZeroShotClassification.load(f"./{EXPORT_PATH}_spark_nlp") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("class")

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

pipeline = Pipeline(stages=[document_assembler, tokenizer, classifier])

text = [["I have a problem with my iphone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to hurry up and get to the airport. We are going to miss our flight!"]]

input_df = spark.createDataFrame(text, ["text"])
model = pipeline.fit(input_df)
model.transform(input_df).select("class.result").show(truncate=False)

+--------+
|result  |
+--------+
|[urgent]|
|[urgent]|
|[urgent]|
|[urgent]|
|[movie] |
|[urgent]|
|[urgent]|
+--------+



That's it! You can now go wild and use hundreds of
`BartForZeroShotClassification` models as zero-shot classifiers from HuggingFace 🤗 in Spark NLP 🚀